In [1]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
import numpy as np
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

<hr />
<hr />
<hr />

In [2]:
results = spark.read.csv('gs://ai-covid19-datalake/trusted/results_google_ai_seminario.csv', header=True)
results.limit(10).toPandas()

,ID,experiment_filter,undersampling_method,undersampling_column,filename,experiment_id,n_covid,n_not_covid,model_name,model_maxIter,model_maxDepth,model_maxBins,model_AUC_ROC,model_AUC_PR,model_covid_precision,model_covid_recall,model_covid_f1,model_not_covid_precision,model_not_covid_recall,model_not_covid_f1,model_avg_precision,model_avg_recall,model_avg_f1,model_avg_acc,model_TP,model_TN,model_FN,model_FP,model_time_exec,model_col_set
0,1,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_1_0,0,76982,44537,GBT,100,7,32,0.949472,0.938372,0.954767,0.979128,0.966794,0.962257,0.919817,0.940558,0.958512,0.949472,0.953676,0.957391,22564,12263,481,1069,"134,3842",cols_set_1
1,2,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_2_0,0,76982,44537,GBT,100,7,32,0.937249,0.924798,0.943801,0.975290,0.959287,0.954477,0.899208,0.926019,0.949139,0.937249,0.942653,0.947478,22655,12035,574,1349,"113,3891",cols_set_2
2,3,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_3_0,0,76982,44537,GBT,100,7,32,0.909478,0.901661,0.915971,0.971585,0.942959,0.945696,0.847371,0.893838,0.930834,0.909478,0.918398,0.925791,22499,11459,658,2064,"113,7004",cols_set_3
3,4,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_1_1,1,76879,44537,GBT,100,7,32,0.949929,0.939886,0.955221,0.980499,0.967695,0.964121,0.919360,0.941208,0.959671,0.949929,0.954452,0.958302,22676,12119,451,1063,"120,3549",cols_set_1
4,5,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_2_1,1,76879,44537,GBT,100,7,32,0.937097,0.926052,0.943127,0.976270,0.959412,0.956185,0.897924,0.926139,0.949656,0.937097,0.942776,0.947612,22586,11981,549,1362,"11,2891",cols_set_2
5,6,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_3_1,1,76879,44537,GBT,100,7,32,0.910786,0.896698,0.919762,0.967668,0.943107,0.938502,0.853905,0.894207,0.929132,0.910786,0.918657,0.926006,22387,11415,748,1953,"115,7130",cols_set_3
6,7,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_1_2,2,75862,44537,GBT,100,7,32,0.949014,0.940978,0.952852,0.980928,0.966686,0.965700,0.917099,0.940772,0.959276,0.949014,0.953729,0.957358,22271,12191,433,1102,"117,8000",cols_set_1
7,8,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_2_2,2,75862,44537,GBT,100,7,32,0.936452,0.922622,0.943707,0.974258,0.958739,0.952419,0.898647,0.924752,0.948063,0.936452,0.941746,0.946703,22481,11890,594,1341,"115,7090",cols_set_2
8,9,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_epiweeks_cols_set_3_2,2,75862,44537,GBT,100,7,32,0.912641,0.899769,0.920529,0.968482,0.943897,0.940731,0.856800,0.896806,0.930630,0.912641,0.920352,0.927312,22124,11428,720,1910,"117,0424",cols_set_3
9,10,PCR,STRAT_SAMP,AGEGRP,strat_samp_pcr_agegrp_cols_set_1_0,0,76847,44537,GBT,100,7,32,0.949269,0.941497,0.952998,0.981190,0.966889,0.966164,0.917347,0.941123,0.959581,0.949269,0.954006,0.957614,22587,12364,433,1114,"15,0444",cols_set_1


In [3]:
float_cols = ['model_AUC_ROC', 'model_AUC_PR', 'model_covid_precision', 'model_covid_recall', 'model_covid_f1', 'model_not_covid_precision', 'model_not_covid_recall', 'model_not_covid_f1', 'model_avg_precision', 'model_avg_recall', 'model_avg_f1', 'model_avg_acc']
for col in float_cols:
    results = results.withColumn(col, F.col(col).cast('float'))

In [9]:
results.groupBy(['model_name', 'experiment_filter','undersampling_method', 'undersampling_column', 'model_col_set']).count()\
       .orderBy(['model_name', 'experiment_filter','undersampling_method', 'undersampling_column', 'model_col_set']).show(100)

+----------+-----------------+--------------------+--------------------+-------------+-----+
|model_name|experiment_filter|undersampling_method|undersampling_column|model_col_set|count|
+----------+-----------------+--------------------+--------------------+-------------+-----+
|       GBT|              LAB|          STRAT_SAMP|            EPIWEEKS|   cols_set_1|    3|
|       GBT|              LAB|          STRAT_SAMP|            EPIWEEKS|   cols_set_2|    3|
|       GBT|              LAB|          STRAT_SAMP|            EPIWEEKS|   cols_set_3|    3|
|       GBT|               NA|              KMODES|              AGEGRP|   cols_set_1|    3|
|       GBT|               NA|              KMODES|              AGEGRP|   cols_set_2|    3|
|       GBT|               NA|              KMODES|              AGEGRP|   cols_set_3|    3|
|       GBT|               NA|                ROSE|                  NA|   cols_set_1|    3|
|       GBT|               NA|                ROSE|                  N

In [14]:
cols = ['model_name', 'experiment_filter','undersampling_method', 'undersampling_column', 'model_col_set']

results.groupBy(cols)\
        .agg({'model_avg_precision': 'mean',
              'model_avg_recall': 'mean',
              'model_avg_f1': 'mean'})\
        .orderBy(cols).toPandas()

,model_name,experiment_filter,undersampling_method,undersampling_column,model_col_set,avg(model_avg_precision),avg(model_avg_recall),avg(model_avg_f1)
0,GBT,LAB,STRAT_SAMP,EPIWEEKS,cols_set_1,0.959289,0.940997,0.949546
1,GBT,LAB,STRAT_SAMP,EPIWEEKS,cols_set_2,0.949524,0.928579,0.938260
2,GBT,LAB,STRAT_SAMP,EPIWEEKS,cols_set_3,0.936615,0.908421,0.921036
3,GBT,NA,KMODES,AGEGRP,cols_set_1,0.934566,0.922365,0.928035
4,GBT,NA,KMODES,AGEGRP,cols_set_2,0.920636,0.907342,0.913464
5,GBT,NA,KMODES,AGEGRP,cols_set_3,0.896842,0.884388,0.890120
6,GBT,NA,ROSE,NA,cols_set_1,0.862574,0.859111,0.859314
7,GBT,NA,ROSE,NA,cols_set_2,0.818233,0.815798,0.815453
8,GBT,NA,ROSE,NA,cols_set_3,0.788703,0.786662,0.786232
9,GBT,PCR,STRAT_SAMP,AGEGRP,cols_set_1,0.959685,0.950098,0.954531


In [ ]:
cols = ['model_name', 'experiment_filter','undersampling_method', 'undersampling_column', 'model_col_set']

results.groupBy(cols)\
       .agg({'model_avg_precision': 'stddev',
              'model_avg_recall': 'stddev',
              'model_avg_f1': 'stddev'})\
        .orderBy(cols).toPandas()